# **Efeito de congestionamentos em acidentes de trânsito**
## **Estudo de caso de São Paulo**

### Notebook 1a - Dados de Acidentes

Neste notebook, são realizados o carregamento e o pré-processamento dos dados de acidentes.

A origem dos dados é o sistema **InfoSiga** do governo do estado de São Paulo 
- `http://catalogo.governoaberto.sp.gov.br/dataset/infosiga-sp-sistema-de-informacoes-gerenciais-de-acidentes-de-transito-do-estado-de-sao-paulo` (acesso em 10/05/2024)

O resultado desse notebook é o arquivo `data/acidentes/df_acidentes.csv`

In [1]:
import pandas as pd
import geopandas as gpd
import folium
import shapely as sh
import numpy as np
import math
import networkx as nx
from datetime import datetime
import difflib
import time
import shapely as sh

import aux.sao_paulo_shapefile as sp_shp
import aux.sao_paulo_graph as sp_graph
import aux.preprocess_congestion as cong
import aux.preprocess_accidents as acc

from scipy.spatial.distance import euclidean
from scipy.spatial import KDTree

# se nenessário recarregar os arquivos auxiliares
from importlib import reload

import requests
import urllib.parse

reload(sp_shp)
reload(sp_graph)
reload(cong)
reload(acc)

# se necessário instalar os pacotes
# !pip install folium

<module 'aux.preprocess_accidents' from 'C:\\Users\\gigec\\OneDrive\\Documentos\\Pedro\\MBA\\TCC\\src\\aux\\preprocess_accidents.py'>

### Carregamento e unificação dos arquivos

Aqui carregamos os dados e fazemos adequações nos conteúdos e tipos das colunas, a fim de unificar os dois arquivos (fatais e não fatais)

In [2]:
colunas = [
    'chave', 
    'data',
    'hora',
    'dia_semana',
    'municipio',
    'logradouro',
    'numero_endereco',
    'latitude',
    'longitude',
    'tipo_acidente',
    'qtde_vitimas',
    'tempo_acidente_obito'
]

colunas_originais = [
    'ID',
    'Data do Acidente',
    'Hora do Acidente',
    'Dia da semana',
    'Município',
    'Logradouro',
    'Numeral / KM',
    'Lat (GEO)',
    'Long (GEO)',
    'Tipo de Acidente',
    'Quantidade de vítimas',
    'Tempo entre o Acidente e as Mortes'
]

In [3]:
#acidentes fatais

df_fatais = pd.read_csv('data/acidentes/acidentes_fatais.csv', sep = ';', encoding = 'ANSI')
df_fatais['Tipo de Acidente'] = 'fatal'
df_fatais = df_fatais[colunas_originais]
df_fatais.columns = colunas

In [4]:
# acidentes não fatais

df_nao_fatais = pd.read_csv('data/acidentes/acidentes_nao_fatais.csv', sep = ';', encoding = 'ANSI')
df_nao_fatais.rename( 
    { 
        'LAT_(GEO)' : 'Lat (GEO)', 
        'LONG_(GEO)' : 'Long (GEO)',
        'Numero/KM' : 'Numeral / KM',
        'Dia da Semana' : 'Dia da semana'
    }, inplace=True, axis=1)

def define_tipo_acidente (acidente):
    if acidente['Tipo de Acidente - Atropelamento (Pedestre)'] == 1:
        return 'nao fatal - atropelamento'
    if acidente['Tipo de Acidente - Atropelamento (Animal)'] == 1:
        return 'nao fatal - atropelamento'
    if acidente['Tipo de Acidente - Choque'] == 1:
        return 'nao fatal - colisao'
    if acidente['Tipo de Acidente - Colisão'] == 1:
        return 'nao fatal - colisao'
    return 'nao fatal - outros'

df_nao_fatais['Tipo de Acidente'] = df_nao_fatais.apply(lambda a : define_tipo_acidente(a), axis = 1) 

df_nao_fatais['Quantidade de vítimas'] = df_nao_fatais[
        ['Pessoas Envolvidas - Grave', 'Pessoas Envolvidas - Leve']].apply(np.sum, axis = 1)

df_nao_fatais['Tempo entre o Acidente e as Mortes'] = None

df_nao_fatais = df_nao_fatais[colunas_originais]
df_nao_fatais.columns = colunas

In [5]:
# unificação dos datasets

df_acidentes = pd.concat([df_fatais, df_nao_fatais]) 

df_acidentes.drop('tempo_acidente_obito', inplace = True, axis = 1) # coluna não é utilizada

### Tratamento de duplicatas

In [6]:

print('Total de acidentes (pré duplicatas): \t', len(df_acidentes))

df_acidentes['chave'] = df_acidentes.apply(
    lambda a : str(a['data']) + str(a['hora']) + str(a['municipio']) + str(a['logradouro']) + str(a['numero_endereco']),
    axis = 1)

df_acidentes.sort_values('qtde_vitimas', inplace = True)
df_acidentes.drop_duplicates('chave', keep = 'last', inplace = True)
df_acidentes.reset_index(inplace = True)

print('Total de acidentes: \t\t\t', len(df_acidentes))

Total de acidentes (pré duplicatas): 	 959753
Total de acidentes: 			 959225


### Valores nulos

In [8]:
# casos sem endereco nem coordenadas, apenas retiramos

df_acidentes = df_acidentes[
    ~((df_acidentes['logradouro'] == 'NAO DISPONIVEL') &
    (df_acidentes['latitude'].isnull()) &
    (df_acidentes['longitude'].isnull()))]

# 23 542 registros deletados (2.5%)
print(len(df_acidentes))

935683


### Busca de coordenadas no OSM

In [9]:
# casos sem coordenadas - buscamos as coordenadas do openstreetmap

geocode_api_url = 'https://nominatim.openstreetmap.org/search.php?q='
def busca_coordenadas(municipio, logradouro, numero):
    endereco = municipio + ', ' + logradouro + ', ' + str(numero)
    endereco = urllib.parse.quote(endereco)
    url = geocode_api_url + endereco + '&format=jsonv2'
    response = requests.get(url)
    if response.status_code != 200 or response.json() == []: 
        return None
        
    lat = float(response.json()[0]['lat'])
    lon = float(response.json()[0]['lon'])
    return (lat, lon) 

In [10]:
sem_coord = df_acidentes[
    (df_acidentes['latitude'].isnull()) &
    (df_acidentes['longitude'].isnull()) &
    (df_acidentes['municipio'] == 'SAO PAULO') &
    (df_acidentes['numero_endereco'] != 'NAO DISPONIVEL')]

n = len(sem_coord)
print(n, ' registros sem coordenadas') # 12138

12138  registros sem coordenadas


In [11]:
sem_coord.sample(3)

,index,chave,data,hora,dia_semana,municipio,logradouro,numero_endereco,latitude,longitude,tipo_acidente,qtde_vitimas
38758,386018,2022-02-0623:00SAO PAULOACESSO AVENIDA PROFESS...,2022-02-06,23:00,DOMINGO,SAO PAULO,ACESSO AVENIDA PROFESSOR JOAO BATISTA CONTI,152,NaN,NaN,nao fatal - outros,0
495887,578548,2021-01-1219:22SAO PAULOVIADUTO MINISTRO ALIOM...,2021-01-12,19:22,TERÇA,SAO PAULO,VIADUTO MINISTRO ALIOMAR BALEEIRO,1,NaN,NaN,nao fatal - colisao,1
69485,758392,2019-12-1418:54SAO PAULOPRESIDENTE DUTRA BR 1161,2019-12-14,18:54,SÁBADO,SAO PAULO,PRESIDENTE DUTRA BR 116,1,NaN,NaN,nao fatal - outros,0


In [13]:
def trata_logradouro(log):
    l = log.replace('ACESSO', '').strip()
    l = l.replace('JACUPESSEGO', 'JACU PESSEGO').strip()
    l = l.replace('CENTRO', '').strip()
    l = l.replace('ESTACAO', 'ESTAÇÃO').strip()
    l = l.replace('ESTAÇÃO METRO', 'ESTAÇÃO').strip()
    return l

In [58]:
print(busca_coordenadas('Sao Paulo', 'Avenida Paulista', ''))

(-23.5570054, -46.6612537)


In [15]:
# esse processamento deve ser feito em pequenos lotes (limite da API do OSM)

j = 0
s = 0
for i, r in sem_coord.iterrows():    
    j += 1
    # use essa linha para controlar os lotes
    if j < 99999: continue
    if (j % 10 == 0):
        print('{}\t{}/{}'.format(datetime.now().strftime('%H:%M:%S'), j, n))
        print(j - s, 'erros\n')
        
    time.sleep(2)
    logradouro = trata_logradouro(r['logradouro'])
    
    numero = r['numero_endereco']
    if 'ESTAÇÃO' in logradouro: numero = ''
    
    coords = busca_coordenadas(r['municipio'], logradouro, numero)
    if coords == None: 
        print(r['municipio'], logradouro, r['numero_endereco'])
        continue
    
    s += 1
    df_acidentes.at[i, 'latitude'] = coords[0]
    df_acidentes.at[i, 'longitude'] = coords[1]

In [57]:
df_acidentes.to_csv('data/acidentes/df_acidentes_backup_prepreenchido.csv')

In [17]:
df_acidentes = pd.read_csv('data/acidentes/df_acidentes_backup_prepreenchido_4.csv')

df_acidentes_final = df_acidentes[~(df_acidentes['longitude'].isnull())]
df_acidentes_final = df_acidentes_final[~(df_acidentes_final['latitude'].isnull())]

df_acidentes_final = df_acidentes_final[df_acidentes_final['latitude'] != 'NAO DISPONIVEL']
df_acidentes_final = df_acidentes_final[df_acidentes_final['longitude'] != 'NAO DISPONIVEL']

df_acidentes_final = df_acidentes_final[df_acidentes_final['municipio'] == 'SAO PAULO']

print(len(df_acidentes_final))
# 216010

df_acidentes = pd.read_csv('data/acidentes/df_acidentes.csv')

216010


### colocando os acidentes no grafo

In [18]:
print('{}\t Fazendo leitura do arquivo (3 min)'.format(datetime.now().strftime('%H:%M:%S')))
gdf = sp_shp.read_and_process_file('data/logradouros/SIRGAS_SHP_logradouronbl_line.shp')
print('{}\t Iniciando criação do grafo (3 min)'.format(datetime.now().strftime('%H:%M:%S')))
G = sp_graph.create_and_process_graph (gdf)
print('{}\t Fim da criação do grafo'.format(datetime.now().strftime('%H:%M:%S')))

16:41:37	 Fazendo leitura do arquivo (3 min)
16:43:19	 Iniciando criação do grafo (3 min)
16:44:07	 Fim da criação do grafo


In [19]:
tree, nodes, edges_of_the_nodes = acc.build_kd_tree(G)
print(len(nodes))
print(len(edges_of_the_nodes))

1220194
1220194


In [37]:
def plot_mapa_encontra_aresta (nodes, edges_of_the_nodes, tree, acidente):
    v, e = acc.find_edge(nodes, edges_of_the_nodes, tree, acidente)
    
    #plota acidente
    v_a = (float(acidente['latitude'].replace(',','.')), float(acidente['longitude'].replace(',','.')))
    
    m = folium.Map(zoom_start=18,location=v_a, tiles="cartodb positron")
    folium.Circle(
        location = v_a,
        radius = 7,
        color = "black",
        weight = 3,
        fill_opacity = 0.7,
        opacity = 1,
        fill_color = "red"
    ).add_to(m)
    
    #plota aresta
    e_data = G.get_edge_data(e[0], e[1])
    g = e_data[0]['geometry']
    d_e = g.distance(sh.Point(v[0], v[1]))
    coords = [(point[1], point[0]) for point in g.coords]  # Revertendo a ordem para (lat, lon)
    folium.PolyLine(coords, color="blue", weight=5, opacity=1).add_to(m)
    
    # plota ponto da aresta
    folium.Circle(
        location = v[::-1],
        radius = 5,
        color = "black",
        weight = 3,
        fill_opacity = .5,
        opacity = 1,
        fill_color = "orange"
    ).add_to(m)
    
    #print('aresta:                  ', e_data)
    print('coordenadas do acidente: ', [round(x, 5) for x in v_a])
    print('coordenadas do vértice:  ', [round(x, 5) for x in v[::-1]])
    print('distancia:               ', d_e)
    display(m)

In [41]:
i = 10000
a = df_acidentes_final.iloc[i]

print(a['logradouro'], a['numero_endereco'])
plot_mapa_encontra_aresta(nodes,edges_of_the_nodes, tree, a)

RUA RUBEM SOUTO DE ARAUJO 666
coordenadas do acidente:  [-23.738, -46.70643]
coordenadas do vértice:   [-23.73799, -46.70642]
distancia:                0.0


In [22]:
print('{}\tIniciando atribuição do acidentes nas arestas (1 min)'.format(datetime.now().strftime('%H:%M:%S')))

acc.put_accidents_on_edges(G, df_acidentes_final, True)

print('{}\tFinalizando atribuição do acidentes nas arestas'.format(datetime.now().strftime('%H:%M:%S')))

16:44:44	Iniciando atribuição do acidentes nas arestas
16:45:43	Finalizando atribuição do acidentes nas arestas
